In [ ]:
import numpy as np
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt


os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def load_and_preprocess_data(data_path, actions, max_len=100):
    all_data = []
    labels = []

    for file_name in os.listdir(data_path):
        if file_name.endswith('.npy'):
            data = np.load(os.path.join(data_path, file_name))
            if data.size == 0:
                continue  # 데이터가 비어있으면 건너뜁니다.

            label = int(file_name.split('_')[-1].split('.')[0])
            labels.append(label)

            # 각 프레임 데이터 패딩 (내부 차원 표준화)
            padded_frames = pad_sequences(data, maxlen=max_len, dtype='float32', padding='post', truncating='post', value=0.0)
            # 전체 시퀀스 길이 패딩
            padded_data = pad_sequences([padded_frames], maxlen=30, dtype='float32', padding='post', truncating='post', value=0.0)
            all_data.append(padded_data.squeeze(0))  # 불필요한 차원 제거

    # 레이블 원-핫 인코딩
    labels = to_categorical(labels, num_classes=len(actions))

    return np.array(all_data), labels


In [ ]:
actions = ['down', 'up', 'prev', 'next']

# 저장된 npy 파일들을 로드하는 코드
# 저장된 npy 파일이 있는 디렉토리 경로
data_path = '../processed_dataset_4'

# 데이터 로드 및 전처리
max_len = 99  # 최대 길이 설정
x_data, y_data = load_and_preprocess_data(data_path, actions, max_len=max_len)

In [ ]:
print(x_data.shape)
print(y_data.shape)

In [ ]:
# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)


In [ ]:
# 모델 구성
model = Sequential([
    LSTM(512, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True),
    Dense(1024, activation='relu'),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Flatten(),
    Dense(len(actions), activation='softmax')
])

# EarlyStopping 콜백 추가
early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1, mode='min', restore_best_weights=True)

# ReduceLROnPlateau 콜백 조정
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, verbose=1, mode='min', min_lr=0.00001)

# ModelCheckpoint 콜백 조정
model_checkpoint = ModelCheckpoint('models/gesture4_fr30_2.keras', monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')


# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 콜백 정의
# callbacks = [
#     ModelCheckpoint('models/updated_model.keras', monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto'),
#     ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=50, verbose=1)
# ]

# 모델 요약
model.summary()

In [ ]:
# 모델 학습
history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=200,
    callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

In [ ]:
# 학습 결과 시각화
fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()
loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
acc_ax.plot(history.history['accuracy'], 'b', label='train acc')
acc_ax.plot(history.history['val_accuracy'], 'g', label='val acc')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuracy')
loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
from tensorflow import keras
model = keras.models.load_model('models/gesture4_fr30_2.keras', compile=False)

model.save('model.h5')

In [ ]:
import numpy as np
from tensorflow import keras
from keras.models import load_model

# 모델 로드
model = load_model('models/gesture4_fr30.keras')

# 모델 요약 출력
model.summary()

# 더미 데이터 생성
dummy_input = np.random.random((1, 30, 512)).astype(np.float32)

# 모델을 테스트하여 출력 노드 이름 확인
try:
    dummy_output = model.predict(dummy_input)
    print("Dummy output generated successfully.")
    output_names = [n.name for n in model.outputs]
    print("Output names:", output_names)
except Exception as e:
    print("Error during model prediction:", e)


In [ ]:
import tensorflow as tf

# 모델 로드
model = tf.keras.models.load_model('models/gesture4_fr30.keras')

# 모델을 SavedModel 형식으로 저장
saved_model_dir = 'saved_model'
model.save(saved_model_dir)

# SavedModel 형식의 모델을 TensorFlow Lite 모델로 변환
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# 변환된 모델을 파일로 저장
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model has been successfully converted to TensorFlow Lite format.")


In [4]:
import tf2onnx
import tensorflow as tf

# Load your existing model
model = tf.keras.models.load_model('path_to_your_model/gesture4_fr30.keras')

# Convert to ONNX
spec = (tf.TensorSpec((None, 30, 99), tf.float32, name="input"),)
output_path = "path_to_save/model.onnx"
model_proto, external_tensor_storage = tf2onnx.convert.from_keras(model, input_signature=spec, output_path=output_path)

TypeError: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 30, 99], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']